In [50]:
import time
import pandas as pd
import numpy as np

start = time.time()

tutors = pd.read_csv("tutors.csv")
students = pd.read_csv("students.csv")
students

,Timestamp,Email Address,Confirm Email,"Student Name - Last, First","Parent / Guardian Name(s) - Last, First",Phone Number - (111) 111 - 1111,School,Grade Level,Instrument (First Choice),Level / Experience,...,Saturday,Sunday,"Does your child have their own instrument at home? (i.e., piano, saxophone, flute, violin etc.)",Tutoring Location,Keyboard Lending Program,End of the Semester Recital,Previous application with TMC,Tutor Matching,Have any of the student's guardians attended a four-year college?,Does the student qualify for the free or reduced lunch program?
0,4/13/2020 11:08:45,ffs@gmail.com,ffs@gmail.com,"Roberts, George","Lewinsky, Monica",(444) 432-4421,Berkeley High School,3-5,Percussion,2,...,"Before 11am, 1pm- 3pm","3pm- 5pm, After 5pm",No,NaN,No,Yes,"No, this is my student's first time applying f...",NaN,Yes,No
1,4/13/2020 11:10:15,jstalin@gmail.com,jstalin@gmail.com,"Stalin, Joseph","Lenin, Vladimir",(650) 679-4564,Berkeley High School,9-12,Flute,0,...,"Before 11am, 11am-1pm, 1pm- 3pm, 3pm- 5pm, Aft...","1pm- 3pm, 3pm- 5pm, After 5pm",Yes,NaN,No,Yes,"Yes, my student has applied for lessons with T...",NaN,No,Yes
2,4/13/2020 11:11:33,gg@gmail.com,gg@gmail.com,"Glover, Donald","Gambino, Childish",(954) 679-7756,Berkeley High SChool,6-8,Bass,3,...,"11am-1pm, 1pm- 3pm, 3pm- 5pm","Before 11am, 11am-1pm, After 5pm",No,NaN,No,Yes,"Yes, my student has previously applied and rec...",NaN,Yes,Yes
3,4/13/2020 11:13:28,yhs@gmail.com,yhs@gmail.com,"Lopez, Jennifer","Cruise, Thomas",(678)321-4352,Berkeley High School,K-2,French horn,0,...,"Before 11am, 11am-1pm, 1pm- 3pm","Before 11am, 11am-1pm, 1pm- 3pm",No,NaN,No,Yes,"Yes, my student has applied for lessons with T...",NaN,No,No
4,4/13/2020 11:14:53,tyg@gmail.com,tyg@gmail.com,"Kardashian, Khloe","Kardashian, Robert",(678) 554-6734,Berkeley High School,6-8,Piano,2,...,"11am-1pm, 1pm- 3pm, 3pm- 5pm","Before 11am, 11am-1pm, 1pm- 3pm, 3pm- 5pm",Yes,NaN,No,No,"No, this is my student's first time applying f...",NaN,No,Yes
5,4/13/2020 11:16:37,tt@gmail.com,tt@gmail.com,"Fiasco, Lupe","3000, Andre",(568) 554-0345,Berkeley High School,6-8,Piano,2,...,"1pm- 3pm, 3pm- 5pm","1pm- 3pm, 3pm- 5pm",No,NaN,No,Yes,"No, this is my student's first time applying f...",NaN,Yes,No
6,4/13/2020 11:28:17,lm@gmail.com,lm@gmail.com,"Mosey, Lil","Black, Kodak",(432)776-9097,Berkeley High,6-8,Piano,2,...,NaN,"1pm- 3pm, 3pm- 5pm",NaN,NaN,No,No,"Yes, my student has previously applied and rec...",NaN,No,Yes
7,4/13/2020 13:47:04,ahmed.baqai@gmail.com,ahmed.baqai@gmail.com,"Clancy, Tom","Clancy, Jacob",6696009181,Cupertino High,9-12,Violin,4,...,"Before 11am, 11am-1pm, 1pm- 3pm, 3pm- 5pm","11am-1pm, 1pm- 3pm, 3pm- 5pm",Yes,NaN,Yes,Yes,"No, this is my student's first time applying f...",NaN,Yes,No
8,4/13/2020 13:48:06,ahmed.baqai@gmail.com,ahmed.baqai@gmail.com,"Baqai, Ali","Baqai, Shahab",6696009181,Hyde Middle School,6-8,Guitar,1,...,"Before 11am, 1pm- 3pm, 3pm- 5pm","11am-1pm, 3pm- 5pm, After 5pm",Yes,NaN,Yes,Yes,"No, this is my student's first time applying f...",NaN,Yes,No
9,4/13/2020 13:49:18,ahmed.baqai@gmail.com,ahmed.baqai@gmail.com,"Brown, Stacy","Brown, Jacob",6696009181,Monta Vista High,9-12,Violin,2,...,"11am-1pm, 1pm- 3pm, 3pm- 5pm, After 5pm","11am-1pm, 1pm- 3pm, 3pm- 5pm, After 5pm",Yes,NaN,Yes,Yes,"Yes, my student has applied for lessons with T...",NaN,Yes,No


In [51]:
# split students into the 4 priorities 
application = 'Previous application with TMC'
freeLunch = 'Does the student qualify for the free or reduced lunch program?'

prevTutored = 'Yes, my student has previously applied and received lessons with TMC'
prevApplied = 'Yes, my student has applied for lessons with TMC before but was not placed for lessons'
newStudent = 'No, this is my student\'s first time applying for lessons with TMC'

students1 = students.loc[(students[application] == prevTutored)]
students2 = students.loc[(students[application] == prevApplied) & (students[freeLunch] == 'Yes')]
students3 = students.loc[(students[application] == prevApplied) & (students[freeLunch] == 'No')]
students4 = students.loc[(students[application] == newStudent) & (students[freeLunch] == 'Yes')]
students5 = students.loc[(students[application] == newStudent) & (students[freeLunch] == 'No')]

In [52]:
# filter out irrelevant columns
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
tutors = tutors[['Name', 'Which instrument(s) would you like to teach?'] + days]
tutors = tutors.rename(columns={'Which instrument(s) would you like to teach?': 'Instrument'})

In [53]:
# remove/replace invalid data entries
tutors = tutors[tutors['Instrument'].notna()]
tutors['Instrument'] = tutors['Instrument'].str.lower()
tutors = tutors.replace(np.nan, '')

In [54]:
# extract day/time availabilities into separate columns
times = ['Before 11am', '11am- 1pm', '1pm- 3pm', '3pm- 5pm', "After 5pm"]

daystimes = [day + ' ' + time for day in days for time in times]
index = 0
for i in range(len(days)):
    for j in range(len(times)):
        d = days[i]
        t = times[j]
        tutors[daystimes[index]] = tutors[d].str.contains(t)
        index += 1

In [55]:
# keep only day, time combination columns in dataframe
tutors = tutors.drop(days, 1)

In [56]:
# add column for number of availabilities
temp = tutors.drop(['Name', 'Instrument'], 1)
temp['Count'] = temp.sum(1)
tutors['Count'] = temp['Count']
tutors = tutors.sort_values(by='Count')

In [57]:
# processed dataframe for tutors
tutors

,Name,Instrument,Monday Before 11am,Monday 11am- 1pm,Monday 1pm- 3pm,Monday 3pm- 5pm,Monday After 5pm,Tuesday Before 11am,Tuesday 11am- 1pm,Tuesday 1pm- 3pm,...,Saturday 11am- 1pm,Saturday 1pm- 3pm,Saturday 3pm- 5pm,Saturday After 5pm,Sunday Before 11am,Sunday 11am- 1pm,Sunday 1pm- 3pm,Sunday 3pm- 5pm,Sunday After 5pm,Count
2,Jose,piano,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,Lizzy,"violin, bass, piano",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
1,Rohan,"flute, piano, guitar",True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
3,zz@gmail.com,"piano, guitar",False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,3
12,gah,viola,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,3
14,help,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,3
10,Bob the Builder,"bass, clarinet",False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,4
11,askfhalkjh,"clarinet, piano",False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,4
13,wat,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4
15,sad,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4


In [60]:
# filter out irrelevant columns
students = students[['Student Name - Last, First', 'Instrument (First Choice)', 'Instrument (Second Choice)', 'Instrument (Third Choice)'] + days]
students = students.rename(columns={'Student Name - Last, First': 'Name'})

In [62]:
# remove/replace invalid data entries
def replace_na(table, key):
    table = table[table[key].notna()]
    table[key] = table[key].str.lower()
    table = table.replace(np.nan, '')
    
replace_na(students, 'Instrument (First Choice)')
replace_na(students, 'Instrument (Second Choice)')
replace_na(students, 'Instrument (Third Choice)')

/Users/shaylandias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [63]:
# extract day/time availabilities into separate columns
index = 0
for i in range(len(days)):
    for j in range(len(times)):
        d = days[i]
        t = times[j]
        students[daystimes[index]] = students[d].str.contains(t)
        index += 1

In [64]:
# add column for number of availabilities
students = students.drop(days, 1)
temp = students.drop(['Name', 'Instrument (First Choice)', 'Instrument (Second Choice)', 'Instrument (Third Choice)'], 1)
temp['Count'] = temp.sum(1)
students['Count'] = temp['Count']
students = students.sort_values(by='Count')

In [65]:
# processed dataframe for students; matches tutors dataframe
students

,Name,Instrument (First Choice),Instrument (Second Choice),Instrument (Third Choice),Monday Before 11am,Monday 11am- 1pm,Monday 1pm- 3pm,Monday 3pm- 5pm,Monday After 5pm,Tuesday Before 11am,...,Saturday 11am- 1pm,Saturday 1pm- 3pm,Saturday 3pm- 5pm,Saturday After 5pm,Sunday Before 11am,Sunday 11am- 1pm,Sunday 1pm- 3pm,Sunday 3pm- 5pm,Sunday After 5pm,Count
21,11,Clarinet,Trombone,NaN,False,False,False,False,False,False,...,False,True,True,True,NaN,NaN,NaN,NaN,NaN,5.0
6,"Mosey, Lil",Piano,NaN,NaN,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,False,False,True,True,False,5.0
18,"A, B",Violin,NaN,NaN,False,False,False,False,True,NaN,...,NaN,NaN,NaN,NaN,True,False,True,True,True,5.0
22,"bob, builder",Flute,NaN,NaN,True,False,False,False,False,NaN,...,NaN,NaN,NaN,NaN,False,False,True,False,False,6.0
19,"cjh, asdkfjh",Flute,Clarinet,Percussion,False,False,False,True,False,True,...,NaN,NaN,NaN,NaN,False,False,False,False,True,6.0
23,"michael, page",Percussion,Violin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,True,False,True,False,False,False,True,7.0
20,"berkelee, berk",Cello,NaN,NaN,True,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,7.0
0,"Roberts, George",Percussion,Flute,NaN,False,False,False,True,False,NaN,...,False,True,False,False,False,False,False,True,True,8.0
2,"Glover, Donald",Bass,Trumpet,Piano,False,False,True,False,False,NaN,...,False,True,True,False,True,False,False,False,True,8.0
28,"Pitt, Bradasdf",Flute,NaN,NaN,True,False,False,True,True,NaN,...,NaN,NaN,NaN,NaN,False,False,False,True,False,8.0


In [66]:
# copy to mutate in function
tutors2 = tutors.copy()

In [73]:
"""
Takes in a student in the form of a tuple with (length, Series) -- values from a df.iterrows() iterable.
Returns a dataframe containing the student as column 0 and every tutor that can be matched as a following column.
Matches are considered valid if there is at least one instrument overlap between student and tutor, and at least one time availability overlap.
"""
def match(student):
    ts = student[1].to_frame()

    t_without_count = tutors2.drop(columns='Count')
    for tutor in t_without_count.iterrows():
        tutor_instruments = tutor[1]['Instrument'].split(', ')
        student_instruments = student[1]['Instrument'].split(', ')
        
        if len(set(tutor_instruments).intersection(set(student_instruments))) > 0 and student[1].add(tutor[1]).value_counts().index.contains(2):
            ts[tutor[1][0]] = tutor[1]
    
    ts = ts.rename(columns={student[0] : student[1][0]})
    return ts[ts[student[1][0]] == True]

In [74]:
# create the iterator from rows of students
s_without_count = students.drop(columns='Count')
students_iter = s_without_count.iterrows()

In [75]:
# create the dataframe to store matches once made
matches = pd.DataFrame(columns=['Student', 'Tutor'])

In [76]:
# iterate through students; use function defined above to get all valid matches
i = 0
for s in students_iter:
    m = match(s)

    names = m.columns
    if (len(names) < 2):
        # not able to match student to any tutors
        matches.loc[i] = [names[0], 'None']
    else:
        # match created: add (student, tutor) pair to matches dataframe and remove tutor from tutors pool
        # currently matches student to tutor with least availabilities; can edit later to add priority consideration
        matches.loc[i] = [names[0], names[1]]
        tutors2 = tutors2.drop(tutors2[tutors2['Name'] == names[1]].index)
    i += 1

KeyError: ('Instrument (First Choice)', 'Instrument (Second Choice)', 'Instrument (Third Choice)')

In [46]:
matches

,Student,Tutor


In [47]:
# unmatched students
matches[matches['Tutor'] == 'None']

,Student,Tutor


In [48]:
# unmatched tutors
tutors2

,Name,Instrument,Monday Before 11am,Monday 11am- 1pm,Monday 1pm- 3pm,Monday 3pm- 5pm,Monday After 5pm,Tuesday Before 11am,Tuesday 11am- 1pm,Tuesday 1pm- 3pm,...,Saturday 11am- 1pm,Saturday 1pm- 3pm,Saturday 3pm- 5pm,Saturday After 5pm,Sunday Before 11am,Sunday 11am- 1pm,Sunday 1pm- 3pm,Sunday 3pm- 5pm,Sunday After 5pm,Count
2,Jose,piano,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
4,Lizzy,"violin, bass, piano",False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,1
1,Rohan,"flute, piano, guitar",True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
3,zz@gmail.com,"piano, guitar",False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,3
12,gah,viola,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,3
14,help,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,3
10,Bob the Builder,"bass, clarinet",False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,4
11,askfhalkjh,"clarinet, piano",False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,4
13,wat,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4
15,sad,guitar,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,4


In [49]:
# time for computation (in seconds):
time.time() - start

192.552729845047